In [1]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import plotly.express as px
# import seaborn as sns

In [3]:
# import csv file
bids = pd.read_csv('./bids.csv')
bidder = pd.read_csv('./train.csv')

In [4]:
bids.head()
# bids.info()
# bidder.info()
# bidder.head()

,bid_id,bidder_id,auction,merchandise,device,time,country,ip,url
0,0,8dac2b259fd1c6d1120e519fb1ac14fbqvax8,ewmzr,jewelry,phone0,9759243157894736,us,69.166.231.58,vasstdc27m7nks3
1,1,668d393e858e8126275433046bbd35c6tywop,aeqok,furniture,phone1,9759243157894736,in,50.201.125.84,jmqlhflrzwuay9c
2,2,aa5f360084278b35d746fa6af3a7a1a5ra3xe,wa00e,home goods,phone2,9759243157894736,py,112.54.208.157,vasstdc27m7nks3
3,3,3939ac3ef7d472a59a9c5f893dd3e39fh9ofi,jefix,jewelry,phone4,9759243157894736,in,18.99.175.133,vasstdc27m7nks3
4,4,8393c48eaf4b8fa96886edc7cf27b372dsibi,jefix,jewelry,phone5,9759243157894736,in,145.138.5.37,vasstdc27m7nks3


## 1. For each auction, the number of unique ip addresses

In [5]:
num_ip = bids.groupby('auction')['ip'].nunique().sort_values(ascending=False)
# save to csv file 
num_ip.to_csv('question_1.csv')
num_ip

auction
jqx39    252681
jefix     97478
du967     41561
opnq4     21980
h1ko2     20119
          ...  
918qe         1
w27e7         1
w272r         1
0ycs7         1
k15l0         1
Name: ip, Length: 15051, dtype: int64

## 2.For each merchandise type, the country which sent the maximum number of bids (in the csv, include also the number of bids sent by that country)


In [7]:
# solution 1
# mc = bids.groupby(['merchandise', 'country']).bid_id.count()
# maxbid_country = mc.unstack().idxmax(axis=1)
# maxbid_country

# double check the answer
# MaxBid_country = {}
# for i in bids.merchandise.unique():
#     MaxBid_country[i] = bids.query('merchandise == @i').groupby('country').bid_id.count().sort_values(ascending=False).index[0]
# print(MaxBid_country)

# solution 2
MerCountry = bids.pivot_table(index='merchandise', columns='country', values='bid_id', aggfunc='count', fill_value=0)
MaxBid_country = MerCountry.idxmax(axis=1)
num_bid = MerCountry.max(axis=1)
MaxBid_country_df = pd.concat([MaxBid_country, num_bid], axis=1)
MaxBid_country_df.columns = ['country', 'num_bid']
MaxBid_country_df.to_csv('question_2.csv')
MaxBid_country_df

,country,num_bid
merchandise,,
auto parts,in,2243
books and music,in,11820
clothing,in,7619
computers,th,16731
furniture,in,47031
home goods,in,169649
jewelry,in,560773
mobile,in,578076
office equipment,in,56140


## 3. The data contains three separate time periods. Calculate average number of unique bidders per auction in each of the three separate time periods

In [8]:
# convert time to timestamp
bids['time'] = pd.to_datetime(bids['time'])

In [9]:
# determine three time periods
bids.time.dt.weekday.value_counts()

3    3105016
2    2872017
4    1679301
Name: time, dtype: int64

In [10]:
from calendar import weekday, day_name

# convert weekday to weekday name
bids['weekday'] = bids.time.apply(lambda x: day_name[weekday(x.year, x.month, x.day)])
bids.head()

,bid_id,bidder_id,auction,merchandise,device,time,country,ip,url,weekday
0,0,8dac2b259fd1c6d1120e519fb1ac14fbqvax8,ewmzr,jewelry,phone0,1970-04-23 22:54:03.157894736,us,69.166.231.58,vasstdc27m7nks3,Thursday
1,1,668d393e858e8126275433046bbd35c6tywop,aeqok,furniture,phone1,1970-04-23 22:54:03.157894736,in,50.201.125.84,jmqlhflrzwuay9c,Thursday
2,2,aa5f360084278b35d746fa6af3a7a1a5ra3xe,wa00e,home goods,phone2,1970-04-23 22:54:03.157894736,py,112.54.208.157,vasstdc27m7nks3,Thursday
3,3,3939ac3ef7d472a59a9c5f893dd3e39fh9ofi,jefix,jewelry,phone4,1970-04-23 22:54:03.157894736,in,18.99.175.133,vasstdc27m7nks3,Thursday
4,4,8393c48eaf4b8fa96886edc7cf27b372dsibi,jefix,jewelry,phone5,1970-04-23 22:54:03.157894736,in,145.138.5.37,vasstdc27m7nks3,Thursday


In [11]:
bidder_per_auction = bids.pivot_table(index='weekday', columns='auction', values='bidder_id', aggfunc='nunique', fill_value=np.nan)
AvgBidder = bidder_per_auction.mean(axis=1)
AvgBidder_df = pd.DataFrame(AvgBidder, columns=['AvgBidder'])
AvgBidder_df.to_csv('question_3.csv')
AvgBidder_df

,AvgBidder
weekday,
Friday,18.909759
Thursday,18.879269
Wednesday,23.316925


In [12]:
print(bids.bidder_id.nunique())
print(bids.auction.nunique())

6614
15051


## 4. For each country, the fraction of times a bid from that country was identified as coming from a robot

In [13]:
# merge two dataframes bidder and bids
bidInfo = pd.merge(bidder, bids, on='bidder_id', how='left')
bidInfo.head()

,bidder_id,payment_account,address,outcome,bid_id,auction,merchandise,device,time,country,ip,url,weekday
0,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,85371.0,yitr4,home goods,phone35,1970-04-23 22:58:09.368421052,ke,82.34.177.248,vasstdc27m7nks3,Thursday
1,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,274487.0,btpyy,home goods,phone1004,1970-04-23 23:10:48.684210526,zm,202.130.71.167,vasstdc27m7nks3,Thursday
2,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,527436.0,kj2ko,home goods,phone4,1970-04-23 23:44:20.578947368,ke,22.54.76.225,vasstdc27m7nks3,Thursday
3,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,554135.0,1m5t8,home goods,phone4,1970-04-23 23:47:10.736842105,ke,32.13.237.126,vasstdc27m7nks3,Thursday
4,91a3c57b13234af24875c56fb7e2b2f4rb56a,a3d2de7675556553a5f08e4c88d2c228754av,a3d2de7675556553a5f08e4c88d2c228vt0u4,0.0,607628.0,btpyy,home goods,phone4,1970-04-23 23:53:09.000000000,ke,202.236.188.240,vasstdc27m7nks3,Thursday


In [14]:
# groupby country and outcome
grouped_co = bidInfo.groupby('country')['outcome'].value_counts(normalize=True)
grouped_co

country  outcome
ad       0.0        0.714286
         1.0        0.285714
ae       0.0        0.914778
         1.0        0.085222
af       0.0        0.955231
                      ...   
zm       1.0        0.029700
zw       0.0        0.984187
         1.0        0.015813
zz       0.0        0.750000
         1.0        0.250000
Name: outcome, Length: 382, dtype: float64

In [15]:
# solution 1
# grouped_co.loc[grouped_co.index.get_level_values(1) == 1]

# solution 2
robot_percentage = grouped_co.loc[:, 1].sort_values(ascending=False)
robot_percentage_df = robot_percentage.to_frame()
robot_percentage_df.reset_index(inplace=True)
robot_percentage_df.columns = ['country', 'percentage']
robot_percentage_df.to_csv('question_4.csv')
robot_percentage_df


,country,percentage
0,tc,1.000000
1,vi,1.000000
2,gi,1.000000
3,gp,1.000000
4,an,1.000000
...,...,...
186,bf,0.004904
187,ne,0.004411
188,gh,0.003577
189,mr,0.003325
